# Authors

- Ikram Kohil, 2019115
- Johnatan Gao, 2013298

# Taint Analysis
In this lab, we will be implementing taint analysis to ensure that all data coming from the outside (user inputs) do not reach critical parts of the system.

# 1. Implementation and tests

In this first part we will implement the teint analysis algorithm called: Possibly Teinted Definitions. We will then test the implementation on four examples in the folder **part_1**.

## 1.1 Possibly Tainted Definitions

According to class notes, there are a couple of variables/concepts that we need to define in order to implement the conceptual algorithm.

$$
in\_taintedDefs: V → \Rho(DEFS)
$$
$$
out\_taintedDefs: V → \Rho(DEFS)
$$

In this statement:

- V: This typically represents a set or type of variables within the context of a programming language or a system. Variables can be anything from integers, strings, objects, etc., depending on the language or system under consideration.
- P(DEFS): P represents the power set, which is the set of all subsets of a given set. DEFS is a set of definitions. So, P(DEFS) would be the power set of the set of definitions. In this context, it suggests that for each variable in V, there is a set of definitions (DEFS) associated with it, and in_taintedDefs maps each variable in V to a subset of its associated definitions.

$$
in\_taintedDefs(v) = \bigcup_{p \in preds(node)} out\_taintedDefs(p)
$$

This equation defines the behavior of the in_taintedDefs function. It states that the in_taintedDefs for a variable v is the union (⋃) of the out_taintedDefs for all predecessors (preds(node)) of a given node. In other words, to compute the in_taintedDefs for a variable at a particular node, you take the union of the out_taintedDefs for all nodes that flow into that node.

$$
tainted\_GEN[node] = 
\left\{
  \begin{array}{ll}
    \ {d_i \in defs[node]  | \exists(d_j, r_k) \in defRefChains | (r_k \in refs[node]), (d_j \in in\_tainted[node])} &\text{ if } (node \in EXPR)\ \\

    \ {d_i \in defs[node]} & \text{ if } (node \in SOURCES) \\

    \emptyset & \text{ if } (node \in FILTERS) \\
    
    \emptyset & \text{ if } (node \in SAFESET)
  \end{array}
\right.
$$

In this statement, we are trying to populate our GEN array. For each node in our CFG, we check for four conditions:
- If the node is part of the FILTERS set, then the tainted_gen for this node is empty
- If the node is part of the SAFESET set, then the tainted_gen for this node is empty as well
- If the node is part of the SOURCES set, then the tainted_gen for this node is the definition for this node
- If the node is part of the EXPR set (in other words, is an expression), then we will:
  - Take the defintion for this node, if it exists
  - Take the definition/reference pair for this node, if it exists
  - Create a pair of (reference, definition), if the reference is part of our reference set and definition is part of the in_tainted set for this node
  
In other words, 
1. Check if node is a definition: First, determine if the current node is a definition.

2. If the node is a definition and the right side is a source: If the node is indeed a definition, and its right side is a source, then mark the definition as tainted. This means that the definition corresponds to a value that is considered tainted or untrusted.

3. If the node is a definition and it corresponds to a filter or safe: If the node is a definition but corresponds to a filter or safe, then no action is taken. This implies that the definition is associated with a value that is considered safe or filtered, so it doesn't need further processing.

4. If the node is a definition and the right side is an expression: If the node is a definition and its right side is an expression, further investigation is needed.
Check if any references in the expression are tainted: Examine all references in the expression associated with the definition. If any of these references are tainted (i.e., marked as untrusted), then the definition is also considered tainted.

$$
tainted\_KILL[node] = {d_k | (var(d_k) = var(d_m)) ^ (d_m \in defs[node] )}
$$

In this statement, we are trying to populate the kill set. Essentially, everytime we detect a definition OR a redefinition, we must add it to our tainted_kill set for this particular node.